# Imports

In [1]:
import numpy as np
import pandas as pd

import datetime

# Config

In [2]:
# Add your input and output path
INPUT_PATH = 'ride_data/'
OUTPUT_PATH = 'data/labels.parquet'

# Loading Dataset

In [3]:
def load_data(path):
    df = pd.read_parquet(path) 
    return df

In [4]:
def pre_processing(dataset, start_date: str, end_date: str):
    # Clean data based on the date
    start_date = datetime.date.fromisoformat(start_date)
    end_date = datetime.date.fromisoformat(end_date)
    clean_dataset = dataset[(dataset['tpep_pickup_datetime'].dt.date >= start_date) &
                            (dataset['tpep_pickup_datetime'].dt.date <= end_date)]
    # Add a new column for pick-up date
    clean_dataset['PU_date'] = pd.to_datetime(clean_dataset['tpep_pickup_datetime'].dt.date)
    return clean_dataset

In [5]:
rides_df = pre_processing(load_data(INPUT_PATH), '2023-01-01', '2023-04-30')  # January --> April

print(f'rides_df shape : {rides_df.shape}')
rides_df.head()

rides_df shape : (12672629, 20)


C:\Users\NPC\AppData\Local\Temp\ipykernel_16604\1519474640.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataset['PU_date'] = pd.to_datetime(clean_dataset['tpep_pickup_datetime'].dt.date)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,PU_date
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,2023-01-01
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,2023-01-01
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,2023-01-01
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,2023-01-01
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,2023-01-01


# Labeling

In [6]:
def labeling(dataset):
    dataset_labels = dataset.groupby(['PULocationID', 'PU_date'])['PU_date'].count().to_frame('Demand')\
        .sort_values(['PULocationID', 'PU_date'], ascending=[True, True]).reset_index()\
        .rename(columns={'PULocationID': 'Location', 'PU_date': 'Date'})
        
    locations = pd.DataFrame(dataset_labels['Location'].unique(), columns=['Location'])
    dates = pd.DataFrame(dataset_labels['Date'].unique(), columns=['Date'])
    # create a DataFrame for all locations and dates
    loc_date_df = locations.merge(dates, how='cross').\
        sort_values(['Location', 'Date'], ascending=[True, True]).reset_index(drop=True)
    # Fill the DataFrame with demands
    labels_df = loc_date_df.merge(dataset_labels, how='left', on=['Location', 'Date']).fillna(value=0)
    
    return labels_df

In [7]:
labels_df = labeling(rides_df)
print(f'labels_df shape : {labels_df.shape}')
labels_df.head()

labels_df shape : (31440, 3)


,Location,Date,Demand
0,1,2023-01-01,40.0
1,1,2023-01-02,31.0
2,1,2023-01-03,27.0
3,1,2023-01-04,7.0
4,1,2023-01-05,15.0


# File saving

In [8]:
def save_labels(dataset, path):
    labels_df = dataset.to_parquet(path, index=False)

In [9]:
save_labels(labels_df, OUTPUT_PATH)